**work in progress**

這篇主要是介紹前幾天寫的`utils`

# Local Settings

In [ ]:
key1 = list(globals().keys())

In [1]:
from learning2read.b04 import *

因為`import *`而額外多出的全域變數們：

In [ ]:
key2 = list(globals().keys())
[x for x in key2 if x not in key1]

# Data Loader

設定Data, File兩個路徑管理器

In [75]:
File = PathMgr('/Users/qtwu/pickle/')
Data = DataMgr('/Users/qtwu/Downloads/data') # htlin's data :p

（由`b04` module產生的函數）  
`Data = PathMgr(PATH_MAC['data'])` $\equiv$ `PathMgr("/Users/qtwu/Downloads/data")`

In [ ]:
# reload modules
import learning2read
from importlib import reload
reload(learning2read)
learning2read.reload_all()

In [ ]:
Data("submit").shape, Data("btest").shape

In [4]:
from learning2read.preprocessing import TotalDataFrame
df_total = TotalDataFrame.run(
    [Data("btrain"), Data("btest"), Data("implicit")])['output']
df_total.sample(10)

/Users/qtwu/anaconda/lib/python3.6/site-packages/learning2read/preprocessing.py:128: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  output = pd.concat(input_data, axis=0, ignore_index=True)


,Book-Rating,ISBN,User-ID
575395,0.0,0307123928,21d8bd3feb
100290,9.0,0836210425,73fee3b799
1086561,0.0,0743444043,682903e3ca
501088,0.0,0590473549,b76f6972a7
724373,0.0,0340778504,6460e84b83
1123013,0.0,0380728273,490b63c716
147218,5.0,1558214267,3911bc1a9c
582167,0.0,0006514219,3e3b54aaf7
84319,8.0,0790009315,492c1cba19
496490,0.0,2070394727,7167497ed7


# HeatMap & Save/Load Pickle

In [7]:
book = df_total.groupby('ISBN').count()['User-ID'].sort_values(ascending=False)
user = df_total.groupby('User-ID').count()['ISBN'].sort_values(ascending=False)

In [8]:
len(book), len(user)

(340556, 105283)

In [10]:
trim_n = 200
book_t = book[:trim_n]
user_t = user[:trim_n]

In [11]:
from collections import defaultdict
is_selected = defaultdict(lambda:False)
for idx_tag in book_t.index:
    is_selected[idx_tag] = True
for idx_tag in user_t.index:
    is_selected[idx_tag] = True

In [16]:
File.is_readable('df.file')

False

In [17]:
# slow
nlod=[]
for r in alod(df_total):
    if is_selected[r['ISBN']] and is_selected[r['User-ID']]:
        nlod.append(r)
df_total_selected = pd.DataFrame(nlod)
df_total_selected=df_total_selected.set_index('index')

save_pickle(File("df_total_selected"),df_total_selected)

# resume
# df_total_selected = load_pickle(File("df_total_selected"))

df_total_selected.sample(3)

,Book-Rating,ISBN,User-ID
index,,,
607343,0.0,0345361792,f4f28f1687
625317,0.0,0345402871,0ef5d109dc
97848,10.0,0060976845,1b6a16e068


In [18]:
from learning2read.plot import HeatMap
obj = HeatMap()
obj.setup(df_total_selected, True)
obj.setup_visualMap(min=-1,max=10)
obj.add('ISBN','User-ID','Book-Rating',
        ftooltip=lambda r:{
            'formatter':'<br>'.join([
                '%.0d'%r['Book-Rating'],
                '%s'%r['ISBN'],
                '%s'%r['User-ID'],
            ])
        })
obj._option['visualMap']={
    'type':'piecewise',
    'pieces': [
        {'gte' : 8, 'lte' : 10, 'color' : 'blue'},
        {'gte' : 5, 'lte' : 7, 'color' : 'yellow'},
        {'gte' : 1, 'lte' : 4, 'color' : 'red'},
        {'value' : -1, 'color' : 'black'},
        {'value' : 0, 'color' : 'grey'},
    ]
}
obj

# Index Control

In [19]:
from learning2read.utils import Index,IndexFold,list_diff

`Index(n, total_n, seed)`

In [20]:
df_total.iloc[Index(5,100,1),:]

,Book-Rating,ISBN,User-ID
17,7.0,0671017241,b295d90f09
72,8.0,0445408073,7bf63153b8
97,9.0,0590407198,d357d9f9c2
8,8.0,0590442384,e61597eacc
32,9.0,0140185232,0f3dd6bffb


`Index(frac, total_n, seed)`

In [21]:
df_total.iloc[Index(0.05,100,1),:]

,Book-Rating,ISBN,User-ID
17,7.0,0671017241,b295d90f09
72,8.0,0445408073,7bf63153b8
97,9.0,0590407198,d357d9f9c2
8,8.0,0590442384,e61597eacc
32,9.0,0140185232,0f3dd6bffb


`IndexFold(k_fold, total_n, seed)`

In [22]:
k_fold = 3
for i_fold in range(k_fold):
    print(df_total.iloc[IndexFold(k_fold,15,1)[i_fold],:])

    Book-Rating        ISBN     User-ID
2           8.0   014028009  02e87fe603
4           9.0  0609600788  98ec1c54b5
6           6.0  0140014748  f1efaa5f51
12          8.0  0425121631  890b95265b
13          5.0  0425132951  6fedd1af91
    Book-Rating        ISBN     User-ID
1          10.0  030796017X  f73f18a13a
3          10.0  0345413350  be36e5b8cb
8           8.0  0590442384  e61597eacc
10         10.0  0060392452  df18d25298
11          7.0  034543479X  21d1766093
    Book-Rating        ISBN     User-ID
0           8.0  1550372289  153d612c88
5           8.0  0786881852  2aed67b99a
7           8.0  0440189993  f113a1da49
9           8.0  0671835823  de1b84707a
14          8.0  0440241065  eea80e2b56


`list_diff(all_set, validation_set)`

In [23]:
i_fold=2
idx_valid = IndexFold(k_fold,15,1)[i_fold]
idx_train = list_diff(range(15), IndexFold(k_fold,15,1)[i_fold])
print(df_total.iloc[idx_valid,:])
print(df_total.iloc[idx_train,:])

    Book-Rating        ISBN     User-ID
0           8.0  1550372289  153d612c88
5           8.0  0786881852  2aed67b99a
7           8.0  0440189993  f113a1da49
9           8.0  0671835823  de1b84707a
14          8.0  0440241065  eea80e2b56
    Book-Rating        ISBN     User-ID
1          10.0  030796017X  f73f18a13a
2           8.0   014028009  02e87fe603
3          10.0  0345413350  be36e5b8cb
4           9.0  0609600788  98ec1c54b5
6           6.0  0140014748  f1efaa5f51
8           8.0  0590442384  e61597eacc
10         10.0  0060392452  df18d25298
11          7.0  034543479X  21d1766093
12          8.0  0425121631  890b95265b
13          5.0  0425132951  6fedd1af91


***

# Procedure Demo

In [25]:
procedure4004 = [
    {
        'class' : 'learning2read.preprocessing.TotalDataFrame',
        'output' : 'df_total',
        'input_data' : ['raw_train', 'raw_test', 'raw_implicit'],
    },
    {
        'class' : 'learning2read.preprocessing.UserRatingSqueeze',
        'output' : 'user_rating',
        'input_data' : 'raw_train',
        'filter_num' : 3,
        'statistics' : ['num','mean','std','skew'],
        'na_policy' : 'median',
    },
    {
        'class' : 'learning2read.preprocessing.BookRatingSqueeze',
        'output' : 'book_rating',
        'input_data' : 'raw_train',
        'filter_num' : 3,
        'statistics' : ['num','mean','std','skew'],
        'na_policy' : 'median',
    },
    {
        'class'  : 'learning2read.preprocessing.UserBookTable',
        'output' : 'df_total_features', # (X,y={-1,0,1,2,...,10})
        'input_data' : ['df_total', 'user_rating', 'book_rating'],
        'na_policy' : None, # should fill it before training
    },
    {
        'class'  : 'learning2read.preprocessing.RowFilter',
        'output' : 'df_train',
        'input_data' : 'df_total_features',
        'func' : r"lambda df : df['Book-Rating']>0",
    },
    {
        'class'  : 'learning2read.b04.LightGBMRandomForest',
        'output' : 'model_rf', # a sklearn like model with (fit,predict)
        'input_data' : 'df_train',
        'param' : {
            'random_state' : 1,
        }
    },
    {
        'class'  : 'learning2read.preprocessing.RowFilter',
        'output' : 'df_test',
        'input_data' : 'df_total_features',
        'func' : r"lambda df : df['Book-Rating']<0",
    },
    {
        'class'  : 'learning2read.submission.Track1',
        'output' : 'track1',
        'input_data' : ['df_test', 'model_rf'],
    },
    {
        'class'  : 'learning2read.submission.Track2',
        'output' : 'track2',
        'input_data' : ['df_test', 'model_rf'],
    },
]

Data Frame View

In [6]:
pd.DataFrame(procedure4004).loc[:,['class','input_data','output']]

,class,input_data,output
0,learning2read.preprocessing.TotalDataFrame,"[raw_train, raw_test, raw_implicit]",df_total
1,learning2read.preprocessing.UserRatingSqueeze,raw_train,user_rating
2,learning2read.preprocessing.BookRatingSqueeze,raw_train,book_rating
3,learning2read.preprocessing.UserBookTable,"[df_total, user_rating, book_rating]",df_total_features
4,learning2read.preprocessing.RowFilter,df_total_features,df_train
5,learning2read.b04.LightGBMRandomForest,df_train,model_rf
6,learning2read.preprocessing.RowFilter,df_total_features,df_test
7,learning2read.submission.Track1,"[df_test, model_rf]",track1
8,learning2read.submission.Track2,"[df_test, model_rf]",track2


# Run Procedure

In [27]:
from learning2read.proc import Procedure
if File.is_readable("p4004_done"):
    P=load_pickle(File("p4004_done"))
else:
    P=Procedure(procedure4004, verbose=True)
    # input data frames
    P.load_data({
        'raw_train': Data('btrain'),
        'raw_test': Data('btest'),
        'raw_implicit': Data('implicit'),
    })

    # run and save
    P.run()
    save_pickle(File("p4004_done"),P)

# outputs
print(P.var.keys())
print(P.var['track1'][:100])

dict_keys(['raw_train', 'raw_test', 'raw_implicit', 'df_total', 'user_rating', 'book_rating', 'df_total_features', 'df_train', 'model_rf', 'df_test', 'track1', 'track2'])
[8, 8, 8, 7, 8, 8, 8, 8, 8, 6, 8, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 7, 8, 8, 8, 8, 8, 8, 8, 7, 7, 8, 8, 8, 8, 8, 8, 8, 9, 8, 8, 8, 8, 9, 8, 8, 8, 8, 8, 9, 8, 8, 7, 8, 8, 8, 8, 8, 8, 9, 8, 8, 9, 9, 9, 9, 7, 6, 8, 7, 8, 8, 8, 8, 8, 9, 8, 8, 8, 8, 8, 8, 8, 8, 9, 8, 8]


# Unpack Procedure.var (very ugly)

In [37]:
# P6 = load_pickle(File('P6_6.pk'))
# globals().update(P6.var)

# Output Submission

In [65]:
t1=pd.DataFrame(P.var['track1'])
t1.to_csv(File('t1.csv'),header=None,index=None)

In [66]:
t1

,0
0,8
1,8
2,8
3,7
4,8
5,8
6,8
7,8
8,8
9,6


***

DEV Code

In [ ]:
# from collections import defaultdict
# class Procedure:
#     def __init__(self,procedure_list,verbose=0):
#         self.procedure_list=procedure_list.copy()
#         self.verbose = verbose
#         self.result_list=[{} for _ in range(len(procedure_list))]
#         self.summay={}
#         self.var={}
#         self.last_done_proc_id=None
#     def append(self,proc):
#         _proc = proc.copy()
#         self.procedure_list.append(_proc)
#         return self
#     def load_data(self,data_dict):
#         for n,v in data_dict.items():
#             self.var[n] = v # just assignment, not ensure it's a hard copy
#         return self
#     def input_from_var(self,vname):
#         if type(vname)==str:
#             return self.var[vname]
#         assert type(vname)==list
#         return [self.var[vn] for vn in vname]
#     def output_to_var(self,vname,data):
#         if type(vname)==str:
#             self.var[vname]=data
#             return
#         assert type(vname)==list
#         assert type(vname)==type(data)
#         assert len(vname)==len(data)
#         for i,d in enumerate(data):
#             self.var[vname[i]]=d
#     def run_id(self,proc_id):
#         proc_dict = self.procedure_list[proc_id]
#         assert proc_dict['class']
#         assert proc_dict['output']
#         assert proc_dict['input_data']
#         run_dict=proc_dict.copy()
#         del run_dict['class']
#         del run_dict['output']
#         run_dict['input_data'] = self.input_from_var(run_dict['input_data'])
        
#         result=eval('%s.run(**run_dict)'%proc_dict['class'])
        
#         if type(proc_dict['output'])==list:
#             assert type(result['output'])==type(proc_dict['output'])
#             assert len(result['output'])==len(proc_dict['output'])
        
#         self.output_to_var(proc_dict['output'], result['output'])
#         del result['output']
#         self.result_list[proc_id] = result
        
#     def run(self):
#         st = (self.last_done_proc_id or -1)+1
#         nproc = len(self.procedure_list)
#         if st==nproc:
#             print("[Procedure] nothing to run.")
#             return self
#         for i in range(st,nproc):
#             if self.verbose:
#                 print("run_id(%d): %s"%(i,str(self.procedure_list[i]) ))
#             self.run_id(i)
#             self.last_done_proc_id=i
#         return self

# import learning2read
# learning2read.reload_all()
# assert learning2read.preprocessing.TotalDataFrame
# assert learning2read.b04.BookVectorPow2AutoEncoder

***

In [ ]:
# import homework
# from homework import *
# reload(homework)
# pass